In [45]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split

In [5]:
!ls /root/EarleeNLP/notebook

5cls.pkl  data-preprocess.ipynb  弹幕清洗7月2号.xlsx


In [4]:
df_phase1 = pd.read_pickle('5cls.pkl')
df_phase1

,text,cls,label
0,什么烂剧 卧槽,辱骂类,3
1,妈的，颠覆三观的绿，看的想吐?,辱骂类,3
2,怪不得你们爱豆越活越娘炮，玩个游戏用力一点就叫的跟狗似的，还是去玩泥巴去吧,辱骂类,3
3,身边有一婊子弄的我实在无奈!怎么对付这种人kao!,辱骂类,3
4,关你球事？你晓得个锤子。,辱骂类,3
...,...,...,...
1751,正经人谁偷窥啊，我都是光明正大的看,色情类,0
1752,我艹，这种波霸给我来10个,色情类,0
1753,屌丝只配性幻想,色情类,0
1754,丝袜美腿拍的好，流量硬币少不了,色情类,0


In [18]:
df_phase2_sexy = pd.read_excel('弹幕清洗7月2号.xlsx', sheet_name='色情', header=None, names=['text'])
df_phase2_sexy['cls'] = '色情类'
df_phase2_sexy['label'] = 0
df_phase2_sexy

,text,cls,label
0,#片,色情类,0
1,#学生妹模特绝对,色情类,0
2,10包射298上门援交,色情类,0
3,13711374005性福就在,色情类,0
4,18禁，未成年人勿入,色情类,0
...,...,...,...
570,公主的胸有点大的,色情类,0
571,公主身材真好，人那么瘦，咪咪那么大,色情类,0
572,黄片地址： http:/*****************,色情类,0
573,具观察，每个女的胸都比较大,色情类,0


In [15]:
df_phase2_ad = pd.read_excel('弹幕清洗7月2号.xlsx', sheet_name='广告', header=None, names=['text'])
df_phase2_ad['cls'] = '广告类'
df_phase2_ad['label'] = 1
df_phase2_ad

,text,cls,label
0,##全球汇集精品##,广告类,1
1,##外围模特预约##,广告类,1
2,#24小时在线#,广告类,1
3,#999条新消息,广告类,1
4,#包夜服务#全天24小时安排,广告类,1
...,...,...,...
1303,做网店 请联系,广告类,1
1304,慕品右淘宝企业店，专注正品宜兴手工紫砂壶，希望给每一位壶友带去一把真正的宜兴紫砂壶，给每一位...,广告类,1
1305,苹果版免费领到vip,广告类,1
1306,各位B友麻烦大家淘宝搜索：***** 并关注店铺一下…谢谢大家！,广告类,1


In [19]:
df_phase2_abuse = pd.read_excel('弹幕清洗7月2号.xlsx', sheet_name='辱骂', header=None, names=['text'])
df_phase2_abuse['cls'] = '辱骂类'
df_phase2_abuse['label'] = 3
df_phase2_abuse

,text,cls,label
0,老子一拳锤通你妈的逼,辱骂类,3
1,:嗯，你是下贱的婊子，没毛病。,辱骂类,3
2,？写得评论被删了？编剧脑子确实有屎啊，怎么了？,辱骂类,3
3,“菜虚鲲”一种鲲，生活在深海里！深海练习时长两年半的个人深海练习生！吃了可以补肾虚的毛病！还...,辱骂类,3
4,(;｀O´)o滚，什么玩意,辱骂类,3
...,...,...,...
2759,左龙暖男牵了个女汉子而且还那么丑真是遗憾了,辱骂类,3
2760,左龙右凤不是第十二期走了吗？怎么第十五期还在？？？？这是为了收视率专门叫回来演给我们看的吗？...,辱骂类,3
2761,作秀，看没人看了炒作,辱骂类,3
2762,做实锤本来就是揭秘真相，照你这么说开挂都是付出努力的，怎么就口嗨了，人家捶地有理有据，像你这...,辱骂类,3


In [20]:
df_phase2_political = pd.read_excel('弹幕清洗7月2号.xlsx', sheet_name='政治', header=None, names=['text'])
df_phase2_political['cls'] = '政治类'
df_phase2_political['label'] = 2
df_phase2_political

,text,cls,label
0,有关六四政治的文章都好压抑,政治类,2
1,六四运动据说闹的很大,政治类,2
2,你知道六四學潮/六四学潮吗,政治类,2
3,八平方事件到底是怎么回事,政治类,2
4,什么是平反六四？,政治类,2
...,...,...,...
758,叶剑英能付出是因为四人帮倒台了。,政治类,2
759,四人帮，江青是唯一的女性，所有泼向女性妖魔化女性的恶毒言辞，都汹涌澎湃直接泼向她,政治类,2
760,始于毛，终于习。,政治类,2
761,真正的天安门事件是1976年四月五日的四五运动！,政治类,2


In [21]:
df_phase2 = pd.concat([df_phase2_sexy, df_phase2_ad, df_phase2_abuse, df_phase2_political])
df_phase2

,text,cls,label
0,#片,色情类,0
1,#学生妹模特绝对,色情类,0
2,10包射298上门援交,色情类,0
3,13711374005性福就在,色情类,0
4,18禁，未成年人勿入,色情类,0
...,...,...,...
758,叶剑英能付出是因为四人帮倒台了。,政治类,2
759,四人帮，江青是唯一的女性，所有泼向女性妖魔化女性的恶毒言辞，都汹涌澎湃直接泼向她,政治类,2
760,始于毛，终于习。,政治类,2
761,真正的天安门事件是1976年四月五日的四五运动！,政治类,2


In [22]:
df_phase1_and_phase2 = pd.concat([df_phase1, df_phase2])
df_phase1_and_phase2

,text,cls,label
0,什么烂剧 卧槽,辱骂类,3
1,妈的，颠覆三观的绿，看的想吐?,辱骂类,3
2,怪不得你们爱豆越活越娘炮，玩个游戏用力一点就叫的跟狗似的，还是去玩泥巴去吧,辱骂类,3
3,身边有一婊子弄的我实在无奈!怎么对付这种人kao!,辱骂类,3
4,关你球事？你晓得个锤子。,辱骂类,3
...,...,...,...
758,叶剑英能付出是因为四人帮倒台了。,政治类,2
759,四人帮，江青是唯一的女性，所有泼向女性妖魔化女性的恶毒言辞，都汹涌澎湃直接泼向她,政治类,2
760,始于毛，终于习。,政治类,2
761,真正的天安门事件是1976年四月五日的四五运动！,政治类,2


In [23]:
df_phase1_and_phase2.groupby('label').count()

,text,cls
label,,
0,827,827
1,1714,1714
2,1429,1429
3,3195,3196


In [26]:
with open('/ai/223/datasets/text_classification/toutiao_cat_data.txt', 'r') as f:
    fl = f.read().splitlines()
    
len(fl)

386163

In [28]:
normal_data = fl[10000:20000]
normal_data[0]

'6552468986791985672_!_109_!_news_tech_!_你回到过去最想见的人是谁？一句话能看出马云的高情商！_!_爱因斯坦,特拉维夫大学,以色列,马云,马克思'

In [29]:
normal_data[1]

'6552418856848064776_!_109_!_news_tech_!_如果八年前花10万元买比特币，现在生活会发生什么变化？_!_'

In [37]:
p = re.compile('.*_!_(.*?)_!_')
# res = p.search(normal_data[0])
# res.groups()[0]

In [38]:
normal_data_cleaned = []
for dt in normal_data:
    if res := p.search(dt):
            normal_data_cleaned.append(res.groups()[0])
len(normal_data_cleaned)

10000

In [39]:
_ = {'text': normal_data_cleaned}
df_phase2_normal = pd.DataFrame(_)
df_phase2_normal['cls'] = '其他类'
df_phase2_normal['label'] = 4
df_phase2_normal

,text,cls,label
0,你回到过去最想见的人是谁？一句话能看出马云的高情商！,其他类,4
1,如果八年前花10万元买比特币，现在生活会发生什么变化？,其他类,4
2,马云：芝麻分又要更新了，芝麻分600分以上大福！你的涨了吗？,其他类,4
3,为什么小米公司成了唯一成功的“小米模式”公司？,其他类,4
4,黑猫：淘宝天猫商家运营如何玩转抽奖精灵工具软件，提高用户粘性,其他类,4
...,...,...,...
9995,经常收到银行的办信用卡的信息，说是白金优质用户，是真的吗？,其他类,4
9996,联想被踢出恒生指数，是什么导致了联想现在的这种境地？,其他类,4
9997,腾讯不需要梦想，但是我们需要,其他类,4
9998,三网融合了，广电网络怎么发展？,其他类,4


In [40]:
df_phase2 = pd.concat([df_phase2, df_phase2_normal])
df_phase2

,text,cls,label
0,#片,色情类,0
1,#学生妹模特绝对,色情类,0
2,10包射298上门援交,色情类,0
3,13711374005性福就在,色情类,0
4,18禁，未成年人勿入,色情类,0
...,...,...,...
9995,经常收到银行的办信用卡的信息，说是白金优质用户，是真的吗？,其他类,4
9996,联想被踢出恒生指数，是什么导致了联想现在的这种境地？,其他类,4
9997,腾讯不需要梦想，但是我们需要,其他类,4
9998,三网融合了，广电网络怎么发展？,其他类,4


In [41]:
df_phase2 = df_phase2.reset_index(drop=True)
df_phase2

,text,cls,label
0,#片,色情类,0
1,#学生妹模特绝对,色情类,0
2,10包射298上门援交,色情类,0
3,13711374005性福就在,色情类,0
4,18禁，未成年人勿入,色情类,0
...,...,...,...
15405,经常收到银行的办信用卡的信息，说是白金优质用户，是真的吗？,其他类,4
15406,联想被踢出恒生指数，是什么导致了联想现在的这种境地？,其他类,4
15407,腾讯不需要梦想，但是我们需要,其他类,4
15408,三网融合了，广电网络怎么发展？,其他类,4


In [44]:
df_phase2.to_pickle('5cls_phase2.pkl')

In [43]:
df_phase2 = df_phase2.dropna()
df_phase2 = df_phase2.reset_index(drop=True)
df_phase2

,text,cls,label
0,#片,色情类,0
1,#学生妹模特绝对,色情类,0
2,10包射298上门援交,色情类,0
3,13711374005性福就在,色情类,0
4,18禁，未成年人勿入,色情类,0
...,...,...,...
15404,经常收到银行的办信用卡的信息，说是白金优质用户，是真的吗？,其他类,4
15405,联想被踢出恒生指数，是什么导致了联想现在的这种境地？,其他类,4
15406,腾讯不需要梦想，但是我们需要,其他类,4
15407,三网融合了，广电网络怎么发展？,其他类,4


In [48]:
df_phase2_train, df_phase2_test = train_test_split(df_phase2, test_size=0.2)
df_phase2_train = df_phase2_train.reset_index(drop=True)
df_phase2_test = df_phase2_test.reset_index(drop=True)

In [50]:
df_phase2_train_only_4 = df_phase2_train.drop(df_phase2_train[df_phase2_train['label']==4].index)

In [52]:
df_phase2_train_only_4 = df_phase2_train_only_4.reset_index(drop=True)

In [53]:
df_phase2_train_only_4.to_pickle('df_phase2_train_only_4.pkl')

In [54]:
df_phase2_test.to_pickle('df_phase2_test.pkl')

In [7]:
with open('/root/hub/golden-horse/data/weiboNER_2nd_conll.train_deseg', 'r') as f:
    fr = f.read()

In [11]:
fr[:200]

'科\tO\n技\tO\n全\tO\n方\tO\n位\tO\n资\tO\n讯\tO\n智\tO\n能\tO\n，\tO\n快\tO\n捷\tO\n的\tO\n汽\tO\n车\tO\n生\tO\n活\tO\n需\tO\n要\tO\n有\tO\n三\tO\n屏\tO\n一\tO\n云\tO\n爱\tO\n你\tO\n\n对\tO\n，\tO\n输\tO\n给\tO\n一\tO\n个\tO\n女\tB-PER.NOM\n人\tI-PER.NOM\n，\tO\n的\tO\n成\tO\n绩\tO\n。\tO\n失\tO\n望\tO\n\n今\tO\n天\tO\n下\tO\n午\tO\n起\t'

In [10]:
fl = fr.splitlines()
fl

['科\tO',
 '技\tO',
 '全\tO',
 '方\tO',
 '位\tO',
 '资\tO',
 '讯\tO',
 '智\tO',
 '能\tO',
 '，\tO',
 '快\tO',
 '捷\tO',
 '的\tO',
 '汽\tO',
 '车\tO',
 '生\tO',
 '活\tO',
 '需\tO',
 '要\tO',
 '有\tO',
 '三\tO',
 '屏\tO',
 '一\tO',
 '云\tO',
 '爱\tO',
 '你\tO',
 '',
 '对\tO',
 '，\tO',
 '输\tO',
 '给\tO',
 '一\tO',
 '个\tO',
 '女\tB-PER.NOM',
 '人\tI-PER.NOM',
 '，\tO',
 '的\tO',
 '成\tO',
 '绩\tO',
 '。\tO',
 '失\tO',
 '望\tO',
 '',
 '今\tO',
 '天\tO',
 '下\tO',
 '午\tO',
 '起\tO',
 '来\tO',
 '看\tO',
 '到\tO',
 '外\tO',
 '面\tO',
 '的\tO',
 '太\tO',
 '阳\tO',
 '。\tO',
 '。\tO',
 '。\tO',
 '。\tO',
 '我\tO',
 '第\tO',
 '一\tO',
 '反\tO',
 '应\tO',
 '竟\tO',
 '然\tO',
 '是\tO',
 '强\tO',
 '烈\tO',
 '的\tO',
 '想\tO',
 '回\tO',
 '家\tO',
 '泪\tO',
 '想\tO',
 '我\tO',
 '们\tO',
 '一\tO',
 '起\tO',
 '在\tO',
 '嘉\tB-LOC.NAM',
 '鱼\tI-LOC.NAM',
 '个\tO',
 '时\tO',
 '候\tO',
 '了\tO',
 '。\tO',
 '。\tO',
 '。\tO',
 '。\tO',
 '有\tO',
 '好\tO',
 '多\tO',
 '好\tO',
 '多\tO',
 '的\tO',
 '话\tO',
 '想\tO',
 '对\tO',
 '你\tO',
 '说\tO',
 '李\tB-PER.NAM',
 '巾\tI-PER.NAM',
 '凡\tI-PER.NAM',
 '想\tO'

In [12]:
import json
a_text = json.load(open('/ai/223/person/lichunyu/datasets/dataf/test/test_A_text.json', 'r'))
a_text

{'NLU00000': {'text': '前天平潭气温多少'},
 'NLU00001': {'text': '快点儿帮我找一下最近的公厕怎么走'},
 'NLU00002': {'text': '中央3套播出的综艺喜乐汇帮我预约播出一下'},
 'NLU00003': {'text': '现在给我定一个周一10点的闹钟'},
 'NLU00004': {'text': '帮我播放正在直播的国际足球赛场-20-21赛季意甲联赛第38轮1'},
 'NLU00005': {'text': '快点查下上海飞往马来西亚的航班哪天有票'},
 'NLU00006': {'text': '你刚才有提醒我去取快递吗？'},
 'NLU00007': {'text': '我想知道洛阳今晚需不需要多加衣物。'},
 'NLU00008': {'text': '感觉昨天的湿度会不会比今天高'},
 'NLU00009': {'text': '今天中午紫外线指数是否会继续上升'},
 'NLU00010': {'text': '能把空调风调低点吗'},
 'NLU00011': {'text': '继续放高尔基的小说在人间的第二章'},
 'NLU00012': {'text': '观看个2019年香港悬疑电影'},
 'NLU00013': {'text': '军事题材电视剧有吗播放一部让我感受一下'},
 'NLU00014': {'text': '立即让我知道西安飞往广州的机票还有几张'},
 'NLU00015': {'text': '帮我看一看订一张最快飞重庆的商务舱机票'},
 'NLU00016': {'text': '脸上长痘痘了要用什么护肤品'},
 'NLU00017': {'text': '组图陈豪出席手机活动周汶不问妹妹感情事'},
 'NLU00018': {'text': '明天天气晴朗吗，想带孩子去花园玩儿'},
 'NLU00019': {'text': '听一听保定人民广播刘聪的节目快点儿给我调'},
 'NLU00020': {'text': '想听英文歌儿了给我播到FM88点7'},
 'NLU00021': {'text': '给我放一下我存在你的所有角落这个小说'},
 'NLU00022': {'text': '我要听北平无战事那个原著'

In [13]:
def normalization(text):
    candidate_num_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    candidate_letter_list = [
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
        'v', 'w', 'x', 'y', 'z',
        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
        'V', 'W', 'X', 'Y', 'Z'
    ]
    for i in candidate_num_list:
        text = text.replace(i, '*')
        
    for i in candidate_letter_list:
        text = text.replace(i, '@')
        
    return text

In [16]:
with open('/ai/223/person/lichunyu/datasets/dataf/test/test_A_text.seq', 'w') as f:
    for idx, info_dict in a_text.items():
        text = normalization(info_dict['text'])
        text = text.replace(' ', '')
        for char in text:
            f.write(char + '\t' + 'O' + '\n')
            
        f.write('\n')

In [17]:
a = {'a': 'a', 'b': 'b'}

In [26]:
d={}
d['a']='A'
d['b']='B'
d['c']='C'
d['1'] = '1'
for k,v in d.items():
    print (k,v)#这个每次运行，print的结果都不一样

a A
b B
c C
1 1
